# pyAstroTrader

## Predict

This notebook is used to predict the future behaviour of the stock prices in the market, using the models created in the ```CreateModel.ipynb``` notebook

First, we need to import the modules that we are going to use:

In [ ]:
import os
import pandas as pd
import dask.dataframe as dd
import numpy as np
import datetime
import plotly.graph_objects as go

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split as ttsplit
from sklearn.metrics import mean_squared_error as mse

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

from IPython.display import display, HTML

import eli5

from pyastrotrader import calculate_chart, calculate_aspects, calculate_transits
from pyastrotrader.utils import create_input_json
from pyastrotrader.constants import *

from settings import *
from helpers import *

We need only to create the dates that we are going to predict as specified in the DAYS_TO_PREDICT configuration variable from settings.py, we will create a list of dictionaries with the correct column name

In [ ]:
today = datetime.datetime.now().date() + datetime.timedelta(days=+1)
days_to_process = []
data_to_dataframe = []
day = today
for day_to_predict in range(DAYS_TO_PREDICT):
    day = day + datetime.timedelta(days=1)
    if day.weekday()==5 or day.weekday()==6:
        continue
    day_as_str = day.strftime('%Y-%m-%d')
    days_to_process.append(day_as_str)
    data_to_dataframe.append({ 'CorrectedDate' : day_as_str})   
    

With the list of dictionaries, we can use the pandas dataframe constructor to create a dataframe with the dates.

In [ ]:
StockPrices = pd.DataFrame(data_to_dataframe)

As we did on the ```CreateModel.ipynb``` notebook, we need to create the natal chart of the selected asset.

In [ ]:
asset_natal_chart_input = create_input_json(NATAL_DATE, 
                                            DEFAULT_PARAMETERS, 
                                            DEFAULT_CONFIG)

asset_natal_chart = calculate_chart(asset_natal_chart_input)
dates_to_generate = days_to_process

Now, for all the dates on the pandas dataframe containing the quotes, we need to generate astrological charts with the list of planets to consider: ```PLANETS_TO_CALCULATE```, their aspects: ```ASPECTS_TO_CALCULATE```

In [ ]:
for current_date in dates_to_generate:
    chart_input = create_input_json(current_date + 'T10:00:00-03:00', 
                                      DEFAULT_PARAMETERS, 
                                      DEFAULT_CONFIG)
    charts[current_date] = calculate_chart(chart_input)
    aspects[current_date] = calculate_transits(asset_natal_chart, charts[current_date], PLANETS_TO_CALCULATE, ASPECTS_TO_CALCULATE, 4)
    aspects_transiting[current_date]= calculate_aspects(charts[current_date], PLANETS_TO_CALCULATE, ASPECTS_TO_CALCULATE, 4)   

We have the natal chart and also all the charts for each date in the pandas dataframe, now we need to add to the pandas dataframe, the astrological aspects that occur in each date, we will set only to 1 if there is a aspect occuring or 0 if not, we also will check for aspects on the transiting chart as well as aspects between the natal chart and the transiting chart

**astro_columns** will indicate the name of the columns containing astrological indicators in the pandas dataframe

In [ ]:
astro_columns = []

for current_planet in PLANETS_TO_CALCULATE:
    column_name="ASTRO_{}_POSITION".format(PLANETS[current_planet]).upper()
    StockPricesDask = dd.from_pandas(StockPrices, npartitions=NPARTITIONS)
    StockPrices[column_name] = StockPricesDask.map_partitions(lambda df : df.apply(lambda x : int(get_degree_for_planet(x, current_planet)), axis =1), meta='int').compute(scheduler='processes')
    StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float', errors='coerce')   
    astro_columns.append(column_name)   
    for second_planet in PLANETS_TO_CALCULATE:
        if current_planet == second_planet:
            continue
        column_name="ASTRO_{}_{}_DIFF".format(PLANETS[current_planet], PLANETS[second_planet]).upper()
        StockPricesDask = dd.from_pandas(StockPrices, npartitions=NPARTITIONS)
        StockPrices[column_name] = StockPricesDask.map_partitions(lambda df : df.apply(lambda x : abs(int(get_degree_for_planet(x, current_planet) - get_degree_for_planet(x, second_planet))), axis =1), meta='int').compute(scheduler='processes')
        StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float', errors='coerce')   
        astro_columns.append(column_name)   
        

for first_planet in PLANETS_TO_CALCULATE:
    for second_planet in PLANETS_TO_CALCULATE:
        for aspect in ASPECTS_TO_CALCULATE:
            column_name="ASTRO_{}_{}_{}".format(PLANETS[first_planet],ASPECT_NAME[aspect],PLANETS[second_planet]).upper()
            aspect_column_name = column_name
            astro_columns.append(column_name)
            StockPricesDask = dd.from_pandas(StockPrices, npartitions=NPARTITIONS)
            StockPrices[column_name] = StockPricesDask.map_partitions(lambda df : df.apply(lambda x : is_aspected(x, first_planet, second_planet, aspect), axis =1), meta='float').compute(scheduler='processes')
            StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float', errors='coerce')

            StockPricesDask = dd.from_pandas(StockPrices, npartitions=NPARTITIONS)
            column_name="ASTRO_TRANSITING_{}_{}_{}".format(PLANETS[first_planet],ASPECT_NAME[aspect],PLANETS[second_planet]).upper()
            astro_columns.append(column_name)
            StockPrices[column_name] = StockPricesDask.map_partitions(lambda df : df.apply(lambda x : is_aspected_transiting(x, first_planet, second_planet, aspect), axis =1), meta='float').compute(scheduler='processes')
            StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float', errors='coerce')                 



We need also to determine which planets are retrograde in each date of the pandas dataframe

In [ ]:
for first_planet in []:
    column_name="ASTRO_{}_RETROGADE".format(PLANETS[first_planet]).upper()
    astro_columns.append(column_name)
    StockPrices[column_name] = StockPrices.apply(lambda x:is_retrograde(x, first_planet), axis =1)
    StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float',errors='coerce')

We need to load the models, which were generated from the ```CreateModel.ipynb``` notebook

In [ ]:
booster_price_change = xgb.Booster()

price_change_model_filename = './output/{}_price_change.model'.format(ASSET_TO_CALCULATE)

booster_price_change.load_model(price_change_model_filename)  # load data

And now, predict the values, using the models created in ```CreateModel.ipynb``` notebook

As we dont need all the columns from the pandas dataframe, we can specify the target variables and the date as the only one that are necessary...

And now save the results in 2 excel files: 1 simplified and another with the entire data.

In [ ]:
output_excel_file='./output/{}.Predict.xlsx'.format(ASSET_TO_CALCULATE)
StockPrices.to_excel(output_excel_file)

In [ ]:
output_excel_file='./output/{}.Predict.Simplified.xls'.format(ASSET_TO_CALCULATE)
CalculatedValue = StockPrices.apply(lambda x:predict_score(x, booster_price_change, StockPrices, astro_columns), axis =1)

if not os.path.isfile(output_excel_file):
    SimplifiedColumns = ['CorrectedDate', 
                         'PredictPriceChange' ]    
    StockPrices['PredictPriceChange'] = CalculatedValue
    StockPrices[SimplifiedColumns].to_excel(output_excel_file)
else:
    column = datetime.datetime.strftime(datetime.datetime.today(), '%Y%m%d%H%M%S') + 'PredictPriceChange'
    StockPrices = pd.read_excel(output_excel_file)
    StockPrices[column] = CalculatedValue
    StockPrices.to_excel(output_excel_file, index=False)
    
    